# LGBM을 활용한 베이스라인

In [1]:
import pandas as pd
import os
import random

## 1. 데이터 로딩

In [2]:
df = pd.read_csv("/opt/ml/input/data/preprocessed_data.csv")
display(df)

,user,assessmentItemID,testId,answer,Timestamp,tag,label,category,test,item,month,day,weekday,hour,elapsed,test_elapsed
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,6,1,1,3,24,1,0,3.0,3.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,6,1,2,3,24,1,0,8.0,8.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,6,1,3,3,24,1,0,7.0,7.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,6,1,4,3,24,1,0,7.0,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,6,1,5,3,24,1,0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,3,71,5,6,5,4,6,65778.0,NaN
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,4,165,1,8,21,4,1,11.0,11.0
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,4,165,2,8,21,4,1,46.0,46.0
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,4,165,3,8,21,4,1,73.0,73.0


## 2. Feature Engineering

In [3]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['user','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('user')['answer'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('user')['answer'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    #유저별 카테고리별
    df['user_category_correct_answer'] = df.groupby(['user', 'category'])['answer'].transform(lambda x: x.cumsum().shift(1))
    df['user_category_total_answer'] = df.groupby(['user', 'category'])['answer'].cumcount()
    df['user_category_acc'] = df['user_category_correct_answer']/df['user_category_total_answer']

    #유저별 태그별
    df['user_tag_correct_answer'] = df.groupby(['user', 'tag'])['answer'].transform(lambda x: x.cumsum().shift(1))
    df['user_tag_total_answer'] = df.groupby(['user', 'tag'])['answer'].cumcount()
    df['user_tag_acc'] = df['user_tag_correct_answer']/df['user_tag_total_answer']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answer'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['tag'])['answer'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    elapsed_user = df.groupby(['user'])['elapsed'].mean()
    elapsed_user.rename('elapsed_user', inplace=True)
    test_elapsed_user = df.groupby(['user'])['test_elapsed'].mean()
    test_elapsed_user.rename('test_elapsed_user', inplace=True)
    elapsed_testId = df.groupby(['testId'])['elapsed'].mean()
    elapsed_testId.rename('elapsed_testId', inplace=True)
    test_elapsed_testId = df.groupby(['testId'])['test_elapsed'].mean()
    test_elapsed_testId.rename('test_elapsed_testId', inplace=True)
    elapsed_category = df.groupby(['category'])['elapsed'].mean()
    elapsed_category.rename('elapsed_category', inplace=True)
    test_elapsed_category = df.groupby(['category'])['test_elapsed'].mean()
    test_elapsed_category.rename('test_elapsed_category', inplace=True)
    elapsed_user_testId = df.groupby(['user', 'testId'])['elapsed'].mean()
    elapsed_user_testId.rename('elapsed_user_testId', inplace=True)
    test_elapsed_user_testId = df.groupby(['user', 'testId'])['test_elapsed'].mean()
    test_elapsed_user_testId.rename('test_elapsed_user_testId', inplace=True)
    elapsed_user_category = df.groupby(['user', 'category'])['elapsed'].mean()
    elapsed_user_category.rename('elapsed_user_category', inplace=True)
    test_elapsed_user_category = df.groupby(['user', 'category'])['test_elapsed'].mean()
    test_elapsed_user_category.rename('test_elapsed_user_category', inplace=True)
    elapsed_assessmentItemID = df.groupby(['assessmentItemID'])['elapsed'].mean()
    elapsed_assessmentItemID.rename('elapsed_assessmentItemID', inplace=True)
    test_elapsed_assessmentItemID = df.groupby(['assessmentItemID'])['test_elapsed'].mean()
    test_elapsed_assessmentItemID.rename('test_elapsed_assessmentItemID', inplace=True)
    elapsed_tag = df.groupby(['tag'])['elapsed'].mean()
    elapsed_tag.rename('elapsed_tag', inplace=True)
    test_elapsed_tag = df.groupby(['tag'])['test_elapsed'].mean()
    test_elapsed_tag.rename('test_elapsed_tag', inplace=True)
    elapsed_user_tag = df.groupby(['user', 'tag'])['elapsed'].mean()
    elapsed_user_tag.rename('elapsed_user_tag', inplace=True)
    test_elapsed_user_tag = df.groupby(['user', 'tag'])['test_elapsed'].mean()
    test_elapsed_user_tag.rename('test_elapsed_user_tag', inplace=True)

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['tag'], how="left")
    df = pd.merge(df, elapsed_user, on=['user'], how="left")
    df = pd.merge(df, test_elapsed_user, on=['user'], how="left")
    df = pd.merge(df, elapsed_testId, on=['testId'], how="left")
    df = pd.merge(df, test_elapsed_testId, on=['testId'], how="left")
    df = pd.merge(df, elapsed_category, on=['category'], how="left")
    df = pd.merge(df, test_elapsed_category, on=['category'], how="left")
    df = pd.merge(df, elapsed_user_testId, on=['user', 'testId'], how="left")
    df = pd.merge(df, test_elapsed_user_testId, on=['user', 'testId'], how="left")
    df = pd.merge(df, elapsed_user_category, on=['user', 'category'], how="left")
    df = pd.merge(df, test_elapsed_user_category, on=['user', 'category'], how="left")
    df = pd.merge(df, elapsed_assessmentItemID, on=['assessmentItemID'], how="left")
    df = pd.merge(df, test_elapsed_assessmentItemID, on=['assessmentItemID'], how="left")
    df = pd.merge(df, elapsed_tag, on=['tag'], how="left")
    df = pd.merge(df, test_elapsed_tag, on=['tag'], how="left")
    df = pd.merge(df, elapsed_user_tag, on=['user', 'tag'], how="left")
    df = pd.merge(df, test_elapsed_user_tag, on=['user', 'tag'], how="left")
    
    return df

In [4]:
df = feature_engineering(df)

In [5]:
pd.set_option('display.max_columns', None)
display(df)

,user,assessmentItemID,testId,answer,Timestamp,tag,label,category,test,item,month,day,weekday,hour,elapsed,test_elapsed,user_correct_answer,user_total_answer,user_acc,user_category_correct_answer,user_category_total_answer,user_category_acc,user_tag_correct_answer,user_tag_total_answer,user_tag_acc,test_mean,test_sum,tag_mean,tag_sum,elapsed_user,test_elapsed_user,elapsed_testId,test_elapsed_testId,elapsed_category,test_elapsed_category,elapsed_user_testId,test_elapsed_user_testId,elapsed_user_category,test_elapsed_user_category,elapsed_assessmentItemID,test_elapsed_assessmentItemID,elapsed_tag,test_elapsed_tag,elapsed_user_tag,test_elapsed_user_tag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,6,1,1,3,24,1,0,3.0,3.0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,0.952667,1429,0.957333,718,6171.22043,50.592824,7022.485333,41.259585,5951.348344,231.106563,3348.666667,7.200000,7150.878613,44.469595,13.660000,13.660000,108.744000,19.895861,3.000000,3.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,6,1,2,3,24,1,0,8.0,8.0,1.0,1,1.000000,1.0,1,1.00,NaN,0,NaN,0.952667,1429,0.917067,3439,6171.22043,50.592824,7022.485333,41.259585,5951.348344,231.106563,3348.666667,7.200000,7150.878613,44.469595,26.112000,26.112000,7416.078667,113.686398,4017.800000,8.250000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,6,1,3,3,24,1,0,7.0,7.0,2.0,2,1.000000,2.0,2,1.00,1.0,1,1.00,0.952667,1429,0.917067,3439,6171.22043,50.592824,7022.485333,41.259585,5951.348344,231.106563,3348.666667,7.200000,7150.878613,44.469595,19.180000,19.180000,7416.078667,113.686398,4017.800000,8.250000
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,6,1,4,3,24,1,0,7.0,7.0,3.0,3,1.000000,3.0,3,1.00,2.0,2,1.00,0.952667,1429,0.917067,3439,6171.22043,50.592824,7022.485333,41.259585,5951.348344,231.106563,3348.666667,7.200000,7150.878613,44.469595,18.076000,18.076000,7416.078667,113.686398,4017.800000,8.250000
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,6,1,5,3,24,1,0,11.0,11.0,4.0,4,1.000000,4.0,4,1.00,3.0,3,1.00,0.952667,1429,0.917067,3439,6171.22043,50.592824,7022.485333,41.259585,5951.348344,231.106563,3348.666667,7.200000,7150.878613,44.469595,35.720000,35.720000,7416.078667,113.686398,4017.800000,8.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,3,71,5,6,5,4,6,65778.0,NaN,1.0,4,0.250000,1.0,4,0.25,1.0,4,0.25,0.666000,999,0.694889,3127,8266.00000,50.000000,7818.508339,388.870539,7109.351951,284.402511,13199.600000,55.000000,13199.600000,55.000000,38109.581940,33704.500000,7430.302067,237.235946,13199.600000,55.000000
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,4,165,1,8,21,4,1,11.0,11.0,1.0,5,0.200000,NaN,0,NaN,NaN,0,NaN,0.652500,783,0.698551,2410,8266.00000,50.000000,8837.122999,488.869757,7265.463398,273.846325,43.333333,43.333333,43.333333,43.333333,247.736667,55.569024,7668.382560,295.900037,43.333333,43.333333
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,4,165,2,8,21,4,1,46.0,46.0,2.0,6,0.333333,1.0,1,1.00,1.0,1,1.00,0.652500,783,0.698551,2410,8266.00000,50.000000,8837.122999,488.869757,7265.463398,273.846325,43.333333,43.333333,43.333333,43.333333,226.533333,27.315436,7668.382560,295.900037,43.333333,43.333333
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,4,165,3,8,21,4,1,73.0,73.0,3.0,7,0.428571,2.0,2,1.00,2.0,2,1.00,0.652500,783,0.698551,2410,8266.00000,50.000000,8837.122999,488.869757,7265.463398,273.846325,43.333333,43.333333,43.333333,43.333333,228.453333,174.050336,7668.382560,295.900037,43.333333,43.333333


## 3. Train/Test 데이터 셋 분리

In [6]:
list(zip(df['user'].value_counts().index, df['user'].value_counts()))[:5]

[(730, 1860), (481, 1847), (1112, 1777), (394, 1774), (926, 1773)]

In [7]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [8]:
random.seed(42)
def train_test_split(df):

    train = df[df["label"] == "train"]
    valid = df[df["label"] == "test"]
    test = valid[valid["answer"] == -1]
    valid = valid[valid["answer"] != -1]
    valid = valid[valid['user'] != valid['user'].shift(-1)]

    return train, valid, test

In [9]:
df.columns

Index(['user', 'assessmentItemID', 'testId', 'answer', 'Timestamp', 'tag',
       'label', 'category', 'test', 'item', 'month', 'day', 'weekday', 'hour',
       'elapsed', 'test_elapsed', 'user_correct_answer', 'user_total_answer',
       'user_acc', 'user_category_correct_answer',
       'user_category_total_answer', 'user_category_acc',
       'user_tag_correct_answer', 'user_tag_total_answer', 'user_tag_acc',
       'test_mean', 'test_sum', 'tag_mean', 'tag_sum', 'elapsed_user',
       'test_elapsed_user', 'elapsed_testId', 'test_elapsed_testId',
       'elapsed_category', 'test_elapsed_category', 'elapsed_user_testId',
       'test_elapsed_user_testId', 'elapsed_user_category',
       'test_elapsed_user_category', 'elapsed_assessmentItemID',
       'test_elapsed_assessmentItemID', 'elapsed_tag', 'test_elapsed_tag',
       'elapsed_user_tag', 'test_elapsed_user_tag'],
      dtype='object')

In [10]:
# 유저별 분리
train, valid, test = train_test_split(df)

# 사용할 Feature 설정
FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
FEATS = df.columns.drop(['answer', "assessmentItemID", "testId", "Timestamp", "label"])
print(len(FEATS), FEATS)

# X, y 값 분리
y_train = train['answer']
train = train.drop(['answer'], axis=1)

y_valid = valid['answer']
valid = valid.drop(['answer'], axis=1)

40 Index(['user', 'tag', 'category', 'test', 'item', 'month', 'day', 'weekday',
       'hour', 'elapsed', 'test_elapsed', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'user_category_correct_answer',
       'user_category_total_answer', 'user_category_acc',
       'user_tag_correct_answer', 'user_tag_total_answer', 'user_tag_acc',
       'test_mean', 'test_sum', 'tag_mean', 'tag_sum', 'elapsed_user',
       'test_elapsed_user', 'elapsed_testId', 'test_elapsed_testId',
       'elapsed_category', 'test_elapsed_category', 'elapsed_user_testId',
       'test_elapsed_user_testId', 'elapsed_user_category',
       'test_elapsed_user_category', 'elapsed_assessmentItemID',
       'test_elapsed_assessmentItemID', 'elapsed_tag', 'test_elapsed_tag',
       'elapsed_user_tag', 'test_elapsed_user_tag'],
      dtype='object')


In [11]:
y_valid

2988       0
3659       1
10859      1
15277      1
23530      0
          ..
2525937    0
2526080    1
2526281    0
2526296    0
2526674    1
Name: answer, Length: 744, dtype: int64

In [12]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

In [13]:
lgb_train = lgb.Dataset(train[FEATS], y_train)
lgb_valid = lgb.Dataset(valid[FEATS], y_valid)

## 4. 훈련 및 검증

In [ ]:
import wandb
from wandb.lightgbm import wandb_callback
wandb.init(project="LGBM", entity="recsys_01", name="fe_40")

model = lgb.train(
    {'objective': 'binary',
    'bagging_fraction': 0.7671,
    'feature_fraction': 0.6808,
    'learning_rate': 0.5731,
    'max_depth': 11,
    'min_child_samples': 272,
    'min_data_in_leaf': 149,
    'n_estimators': 266,
    'num_iterations': 1230,
    'num_leaves': 103
    }, 
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    verbose_eval=100,
    num_boost_round=500,
    early_stopping_rounds=55, 
    callbacks=[wandb_callback()]
)

preds = model.predict(valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

### 4-1. wandb sweep 활용

In [14]:
import wandb
from wandb.lightgbm import wandb_callback

sweep_config = {
  "name" : "[LGBM] sweep",
  "method" : "bayes",
  "metric" : {
      "goal" : "minimize",
      "name" : "val_loss"
  },
  "parameters" : {
    "early_stopping_round" :{
      "min": 50,
      "max": 200,
      "distribution": "int_uniform"
    },
    "learning_rate" :{
      "min": 0.3,
      "max": 1,
      "distribution": "uniform"
    },
    "max_depth" :{
      "min": 5,
      "max": 30,
      "distribution": "int_uniform"
    },
    "n_estimators" :{
      "min": 50,
      "max": 350,
      "distribution": "int_uniform"
    },
    "min_child_samples" :{
      "min": 10,
      "max": 400,
      "distribution": "int_uniform"
    },
    "num_iterations" :{
      "min": 500,
      "max": 1500,
      "distribution": "int_uniform"
    },
    "num_leaves" :{
      "min": 50,
      "max": 250,
      "distribution": "int_uniform"
    },
    "bagging_fraction" :{
      "min": 0.5,
      "max": 1.0,
      "distribution": "uniform"
    },
    "feature_fraction" :{
      "min": 0.5,
      "max": 1.0,
      "distribution": "uniform"
    },
    # "max_bin" :{
    #   "min": 10,
    #   "max": 300,
    #   "distribution": "int_uniform"
    # },
    "min_data_in_leaf" :{
      "min": 10,
      "max": 150,
      "distribution": "int_uniform"
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project="LGBM", entity="recsys_01")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: hhobp47x
Sweep URL: https://wandb.ai/recsys_01/LGBM/sweeps/hhobp47x


In [15]:
# load data
def train():

  wandb.init(project="LGBM", entity="recsys_01")
  config = wandb.config

  # fit model on train
  model = lgb.train(
    {
      'objective': "binary",
      'learning_rate': config.learning_rate,
      'max_depth': config.max_depth,
      'n_estimators': config.n_estimators,
      'min_child_samples': config.min_child_samples,
      'num_iterations': config.num_iterations,
      'num_leaves': config.num_leaves,
      'bagging_fraction': config.bagging_fraction,
      'feature_fraction': config.feature_fraction,
      # 'max_bin': config.max_bin,
      'min_data_in_leaf': config.min_data_in_leaf,
      'feature_pre_filter': False
    },
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    verbose_eval=100,
    num_boost_round=500,
    early_stopping_rounds=config.early_stopping_round,
    callbacks=[wandb_callback()]
)

  # make predictions on valid
  y_pred = model.predict(valid[FEATS])

  # evaluate predictions
  acc = accuracy_score(y_valid, np.where(y_pred >= 0.5, 1, 0))
  auc = roc_auc_score(y_valid, y_pred)

  print(f'VALID AUC : {auc} ACC : {acc}\n')
  wandb.log({"auc": auc, "acc": acc})

In [16]:
wandb.agent(sweep_id, train) # count : 실험 횟수

wandb: Agent Starting Run: soxtfgb2 with config:
wandb: 	bagging_fraction: 0.8360787048117926
wandb: 	early_stopping_round: 50
wandb: 	feature_fraction: 0.8850008639882398
wandb: 	learning_rate: 0.7820590792837139
wandb: 	max_depth: 18
wandb: 	min_child_samples: 281
wandb: 	min_data_in_leaf: 25
wandb: 	n_estimators: 291
wandb: 	num_iterations: 573
wandb: 	num_leaves: 85
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: recsys_01 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback v

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=281 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=281 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Info] Number of positive: 1483205, number of negative: 783381


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_child_samples' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_iterations' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'early_stopping_round' was locked by 'sweep' (ignored update).


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047438 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7676
[LightGBM] [Info] Number of data points in the train set: 2266586, number of used features: 40
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=281 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654378 -> initscore=0.638341
[LightGBM] [Info] Start training from score 0.638341
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	training's binary_logloss: 0.455356	valid_1's binary_logloss: 0.523008
VALID AUC : 0.821730320475776 ACC : 0.7405913978494624




...

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback v

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=281 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=348 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=281 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=348 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=348 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Info] Number of positive: 1483205, number of negative: 783381


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_child_samples' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_iterations' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'early_stopping_round' was locked by 'sweep' (ignored update).


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.056652 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7676
[LightGBM] [Info] Number of data points in the train set: 2266586, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654378 -> initscore=0.638341
[LightGBM] [Info] Start training from score 0.638341
Training until validation scores don't improve for 129 rounds
[100]	training's binary_logloss: 0.432366	valid_1's binary_logloss: 0.512334
Early stopping, best iteration is:
[43]	training's binary_logloss: 0.449611	valid_1's binary_logloss: 0.495254
VALID AUC : 0.8390232840537515 ACC : 0.7580645161290323




wandb: Ctrl + C detected. Stopping sweep.
wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 1874, in _atexit_cleanup
    self._on_finish()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 1999, in _on_finish
    time.sleep(0.1)
Exception


## 5. Inference

In [ ]:
_ = lgb.plot_importance(model)

In [ ]:
# test_df
display(test)

In [ ]:
len(FEATS)

In [ ]:
# MAKE PREDICTION
total_preds = model.predict(test[FEATS])

In [ ]:
# SAVE OUTPUT
output_dir = '/opt/ml/'
write_path = os.path.join(output_dir, "submission_lgbm_fe_40_220502.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))